In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import math
import numpy as np

ROW_LENGTH = 62
COLUMN_LENGTH = 1.25
NUMBER_OF_ROWS = 8

In [2]:
document_topic_probabilities = pd.read_pickle('../pickles/document_topic_probabilities.pickle')
document_topic_probabilities = pd.read_pickle('../pickles/document_topic_probabilities_rogers_gibbs.pickle')
document_topic_probabilities = pd.read_pickle('../pickles/document_topic_probabilities_python.pickle')
document_topic_probabilities = pd.read_pickle('../pickles/document_topic_probabilities_ari_gibbs.pickle')
document_topic_probabilities = pd.read_pickle('../pickles/document_topic_probabilities_temp.pickle')
print document_topic_probabilities[0:10]
document_ids_map = pd.read_pickle('../pickles/doc_id_mapping.pickle').to_dict()
inverted_document_ids_map = {v: int(k) for k, v in document_ids_map.iteritems()}

[[  6.74811022e-03   1.07259449e-04   2.18496594e-04   1.91411539e-01
    1.68854993e-01   2.80181040e-03   9.61263151e-02   3.99250789e-01
    9.79277747e-02   3.65529127e-02]
 [  1.12104526e-02   5.07551087e-04   3.04275203e-04   2.07645102e-01
    1.58832907e-01   4.27657681e-03   7.88493706e-02   3.89151335e-01
    1.19839272e-01   2.93831574e-02]
 [  9.06054140e-03   6.00162603e-05   3.89587030e-04   1.07513241e-01
    2.18925918e-01   1.56639539e-03   5.52335150e-02   3.99790306e-01
    1.12697767e-01   9.47627133e-02]
 [  9.85114753e-03   5.44184934e-04   2.42266768e-04   1.40526191e-01
    2.66197025e-01   9.88576697e-03   6.09143941e-02   3.39005910e-01
    8.32210670e-02   8.96120457e-02]
 [  2.55858435e-02   2.84018060e-03   9.50423090e-03   1.20419344e-01
    1.90182884e-01   3.74275323e-02   8.17770333e-02   3.36374025e-01
    1.15433306e-01   8.04556207e-02]
 [  1.20876017e-02   2.66308958e-04   2.73553844e-03   1.30019784e-01
    1.87174103e-01   6.05152184e-03   6.15144

In [3]:
def calculate_the_sizes(data, number_of_rows, row_length, column_length):
    data_size = len(data)
    single_scan_size = (data_size + 1) / number_of_rows
    column_size = int(math.floor(single_scan_size \
    * (column_length / (row_length + column_length))))
    row_size = single_scan_size - column_size 
    return data_size, single_scan_size, column_size, row_size

In [4]:
def generate_coordinates(data_size, column_size, row_size):    
    x_coord = 0
    y_coord = NUMBER_OF_ROWS - 1
    row_scan = True
    scanning_right = True
    coordinates = {}
    for j in range(0, NUMBER_OF_ROWS):
        for i in range(0, row_size):
            doc_id = i + column_size * j + row_size * j
            coordinates[doc_id] = {'x': x_coord, 'y': y_coord}
            # Switching horizontal scanning to vertical.
            if (i % (row_size - 1) == 0 and i != 0):
                y_coord -= 1
                continue
            if scanning_right:
                x_coord += 1
            else:
                x_coord -= 1         
        # Reversing the scanner's direction.
        scanning_right = not scanning_right
    return coordinates

In [5]:
data_size, single_scan_size, column_size, row_size = \
    calculate_the_sizes(inverted_document_ids_map, NUMBER_OF_ROWS, ROW_LENGTH, COLUMN_LENGTH)
coordinates_mapping = generate_coordinates(data_size, column_size, row_size)

In [8]:
topic_id = 0

probability_matrix = np.zeros(shape=(NUMBER_OF_ROWS, row_size))
for i, doc in enumerate(document_topic_probabilities):
    doc_id = inverted_document_ids_map[i]
#     doc_id = i
    if doc_id in coordinates_mapping:
        doc_coord = coordinates_mapping[doc_id]
        probability_matrix[doc_coord['y']][doc_coord['x']] = doc[topic_id]

In [9]:
import plotly.plotly as py
import plotly.graph_objs as go

x_vector = np.zeros(shape=(row_size, 1))
for i in range(0, row_size):
    x_vector[i] = i
y_vector = np.zeros(shape=(NUMBER_OF_ROWS, 1))
for i in range(0, NUMBER_OF_ROWS):
    y_vector[i] = i

layout = go.Layout(
    title='\'Topic %s\' probability distribution.' % topic_id,
    height=350,
    xaxis=dict(title='x'),
    yaxis=dict(title='y')
)
data = [
    go.Heatmap(
        z=probability_matrix,
        x=x_vector,
        y=y_vector,
        colorscale='Electric',
        colorbar=dict(title='Likelihood')
    )
]
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='topic_%s' % topic_id)